In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.widgets import Slider
from mpl_toolkits.mplot3d import Axes3D
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [2]:
hand_numbers =[np.load(f"hand_number/{n}.npy") for n in range(6)]

for i, n in enumerate(hand_numbers):
    print(i, n.shape)

0 (192, 21, 3)
1 (299, 21, 3)
2 (276, 21, 3)
3 (218, 21, 3)
4 (217, 21, 3)
5 (256, 21, 3)


In [3]:
def plot_landmarks(landmarks):
    landmarks = np.array([l - l.mean(axis=0) for l in landmarks])
    fig = plt.figure(figsize=(6, 4))
    gs = fig.add_gridspec(12, 1)
    ax = fig.add_subplot(gs[:-2, :], projection="3d")
    ax_slider = fig.add_subplot(gs[-1, :])
    slider = Slider(ax_slider, "index", 0, len(landmarks), valinit=0, valstep=1)

    scatters = [ax.scatter3D(*landmarks[idx, :, :].T) for idx in range(len(landmarks))]

    def update(v):
        for s in [s for s in scatters if s.get_visible()]:
            s.set_visible(False)
        idx = int(slider.val)
        scatters[idx].set_visible(True)

    slider.on_changed(update)
    update(None)

    ax.set(xlim=(-0.2, 0.2), ylim=(-0.4, 0.4), zlim=(0, 1), xlabel="x", ylabel="y", zlabel="z")
    ax.view_init(elev=95, azim=90)
    fig.tight_layout()


plot_landmarks(hand_numbers[4])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
hands = np.array([h - h.mean(axis=0) for h in np.concatenate(hand_numbers)])
hands = np.repeat(hands, 2, axis=0)
hands[: hands.shape[0] // 2, 0] = -hands[: hands.shape[0] // 2, 0]

labels = np.concatenate([np.array([i] * len(hn)) for i, hn in enumerate(hand_numbers)])
labels = np.repeat(labels, 2, axis=0)

hands_train, hands_test, labels_train, labels_test = train_test_split(hands, labels)

In [5]:
model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Flatten(input_shape=(21, 3)),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(6, activation="softmax"),
    ]
)
model.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
model.fit(hands_train, labels_train, epochs=100, verbose=0)
model.evaluate(hands_test, labels_test)
# model.save("hand_number_tf")

23/23 [==============================] - 0s 757us/step - loss: 7.5735e-05 - accuracy: 1.0000


[7.573469338240102e-05, 1.0]

In [6]:
hands = np.array([h - h.mean(axis=0) for h in hand_numbers[4]])

model = tf.keras.models.load_model("hand_number_tf")
model.predict(hands[:5])

array([[5.29633830e-08, 3.54650069e-08, 2.40578771e-07, 2.87810490e-05,
        9.99810159e-01, 1.60769399e-04],
       [5.20265502e-08, 3.42807027e-08, 2.37439323e-07, 2.90436074e-05,
        9.99831557e-01, 1.39103606e-04],
       [4.96563501e-08, 3.47361393e-08, 2.14438828e-07, 2.86362847e-05,
        9.99838352e-01, 1.32672591e-04],
       [4.54526976e-08, 1.21419541e-08, 1.16260729e-07, 1.48063955e-05,
        9.99948144e-01, 3.68229921e-05],
       [8.18675332e-08, 2.57151669e-07, 5.91875164e-07, 9.48774323e-05,
        9.98648107e-01, 1.25602540e-03]], dtype=float32)